Analysis of excision using the flank positions identified by blast in notebook 08.

In [1]:
library(tidyverse)
library(cowplot)
library(latex2exp)
library(gggenes)
library(IRkernel)
options(pillar.print_max = 10)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




In [2]:
data_dir <- "exp/multiple_runs11"
is_pos_in_ref <- read_csv(file.path(data_dir, "export", "classify_IS_events", "IS_positions_in_ref_genome.csv"))
depth_df <- read_csv(file.path(data_dir, "export", "classify_IS_events", "depth_interval.csv"))

Rows: 6357 Columns: 12
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): flank, genome, position_status, sv, Line
dbl (5): pos_id, pos, cluster_id, insert_id, Gen
lgl (2): IS_strand, flankMatchDir

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 905 Columns: 11
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): line
dbl (10): group, start, end, depth, length, start_fix, end_fix, is_start, is...

ℹ Use `spec()` to retrieve the full

# Analysis using Excision pipeline
Here, I analyze using the mapping of flanking sequences of ISs in predecessor genomes to the ancestor sequences.

In [3]:
excision_df <- read_csv(file.path(data_dir, "export", "classify_IS_events", "excision_flank_blast.csv"))
excision_df %>% head

Rows: 2863 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): Line, dir, read_name, sseqid
dbl (10): Gen, is_cluster_id, qstart, qend, sstart, send, length, evalue, bi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Line,Gen,is_cluster_id,dir,read_name,sseqid,qstart,qend,sstart,send,length,evalue,bitscore,pident
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
L01-1,1,0,head,L01-1.1.0.dir,L01-1_genome.2_m,1,299,476607,476905,299,2.19e-154,540,100.000
L01-1,1,0,tail,L01-1.1.0.tail,L01-1_genome.2_m,1,299,479999,480297,299,2.19e-154,540,100.000
L01-1,1,2,head,L01-1.1.2.dir,L01-1_genome.2_m,1,298,1185656,1185953,298,7.63e-154,538,100.000
L01-1,1,2,tail,L01-1.1.2.tail,L01-1_genome.2_m,2,299,1203686,1203389,298,7.63e-154,538,100.000
L01-1,1,2,tail,L01-1.1.2.tail,L01-1_genome.2_m,1,222,1086662,1086441,222,7.19e-15,77,67.568
L01-1,1,3,head,L01-1.1.3.dir,L01-1_genome.2_m,1,299,1406056,1406354,299,2.19e-154,540,100.000


In [4]:
excision_stat_df <- read_csv(file.path(data_dir, "export", "classify_IS_events", "excision_status.3000.csv"))
excision_stat_df %>% head

Rows: 1260 Columns: 13
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Line
dbl (11): Gen, is_cluster_id, min_dist, head_qend, tail_qstart, min_i, min_j...
lgl  (1): dir

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Line,Gen,is_cluster_id,min_dist,head_qend,tail_qstart,min_i,min_j,raw_dist,min_start_raw,min_end_raw,flank_dist,dir
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
L01-1,1,0,3093,2999,1,0,1,3094,476905,479999,3093,TRUE
L01-1,1,1,8024,2063,311,2,3,9269,547577,555600,-8024,FALSE
L01-1,1,2,17732,2998,2,4,9,17735,1185953,1203686,17732,FALSE
L01-1,1,3,3093,2999,1,11,12,3094,1406354,1409448,3093,TRUE
L01-1,1,4,4663,2999,1,15,16,4664,1500452,1505116,4663,TRUE
L01-1,1,5,4664,2999,1,17,20,4665,1993862,1998527,4664,TRUE


In [5]:
excision_stat_df %>% select(-min_i, -min_j) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # remove this because pass 8 and pass 20 diverged a lot
	filter(min_dist < 3000) %>%
	head

Line,Gen,is_cluster_id,min_dist,head_qend,tail_qstart,raw_dist,min_start_raw,min_end_raw,flank_dist,dir
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
L01-3,2,10,7,2998,1,-5,1932254,1932248,-7,TRUE
L01-4,1,1,7,2998,1,-5,557155,557149,-7,TRUE
L02-1,2,7,7,2998,1,-5,2097939,2097933,-7,TRUE
L02-3,2,4,7,2998,1,-5,1305532,1305526,-7,TRUE
L02-4,2,7,6,2998,1,-4,2173358,2173353,-6,TRUE
L02-4,2,20,7,2999,1,-6,3667801,3667795,-7,TRUE


In [6]:
is_pos_in_ref %>%
	filter(genome == "Query") %>%
	select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
	pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)) %>%
	head

is_cluster_id,Line,Gen,position_status_r,position_status_f,sv_r,sv_f,flank_pos_r,flank_pos_f
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
0,L01-1,2,original,original,pristine,pristine,476930,480019
1,L01-1,2,new,new,unknown,unknown,552533,560570
2,L01-1,2,new,new,simple_insertion,simple_insertion,711472,711460
3,L01-1,2,new,original,unknown,unknown,1205566,1187842
4,L01-1,2,original,new,unknown,unknown,1192499,1205554
5,L01-1,2,original,original,pristine,pristine,1408214,1405125


In [7]:
excision_stat_df %>% select(Line:min_dist, flank_dist, min_start_raw, min_end_raw, dir) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # fixed 20240703 
	filter(min_dist < 3000) %>% # does not contain full-length ISs
	left_join(
		is_pos_in_ref %>%
		filter(genome == "Query") %>%
		select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
		pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)),
		by = c("Line", "Gen", "is_cluster_id")
	)

Line,Gen,is_cluster_id,min_dist,flank_dist,min_start_raw,min_end_raw,dir,position_status_r,position_status_f,sv_r,sv_f,flank_pos_r,flank_pos_f
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
L01-3,2,10,7,-7,1932254,1932248,TRUE,new,new,simple_insertion,simple_insertion,1933692,1933679
L01-4,1,1,7,-7,557155,557149,TRUE,NA,NA,NA,NA,NA,NA
L02-1,2,7,7,-7,2097939,2097933,TRUE,new,new,simple_insertion,simple_insertion,2139573,2139560
L02-3,2,4,7,-7,1305532,1305526,TRUE,new,new,simple_insertion,simple_insertion,1326211,1326198
L02-4,2,7,6,-6,2173358,2173353,TRUE,new,new,simple_insertion,simple_insertion,2157278,2157266
L02-4,2,20,7,-7,3667801,3667795,TRUE,new,new,simple_insertion,simple_insertion,3534905,3534893
L03-3,1,8,1875,1875,3923798,3925674,TRUE,NA,NA,NA,NA,NA,NA
L03-3,2,6,7,-7,1617303,1617297,TRUE,new,new,simple_insertion,simple_insertion,1578194,1578207
L03-3,2,9,4,-4,1894162,1894165,FALSE,original,original,unknown,unknown,1878154,1878155


### STAT: number of simple excisions

In [8]:
excision_stat_df.stat <- excision_stat_df %>% select(Line:min_dist, flank_dist, min_start_raw, min_end_raw, dir) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # fixed 20240703 
	filter(min_dist < 3000) %>%
	left_join(
		is_pos_in_ref %>%
		filter(genome == "Query") %>%
		select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
		pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)),
		by = c("Line", "Gen", "is_cluster_id")
	) %>% filter(dir, min_dist<100) %>%  # I excluded non-matching dir as this indicates potential inversions
	group_by(Gen) %>% summarise(cnt = n()) %>% ungroup()
excision_stat_df.stat

Gen,cnt
<dbl>,<int>
1,5
2,20


The identity of excluded ones. \\
L03-3	1	8: deletion of both ends \\
L03-3	2	9	4	: deletion bringing an inverted duplicated region close to a distant region

## STAT: Binomtest 8 vs 20
The number of excisions differ much between the two.
The expected ratio should be
44 lines * 8 passages / (43 lines * 12 passages + 44 lines * 8 passages) 
Out of the excisions (attempts) is the losses up until the eighth passage siginificant?

In [9]:
binom.test(excision_stat_df.stat$cnt[1], sum(excision_stat_df.stat$cnt), 44*8/(43*12 + 44*8), alternative="less")


	Exact binomial test

data:  excision_stat_df.stat$cnt[1] and sum(excision_stat_df.stat$cnt)
number of successes = 5, number of trials = 25, p-value = 0.02588
alternative hypothesis: true probability of success is less than 0.40553
95 percent confidence interval:
 0.0000000 0.3754051
sample estimates:
probability of success 
                   0.2 


In [10]:
excision_stat_df %>% select(Line:min_dist, flank_dist, min_start_raw, min_end_raw, dir) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # fixed 20240703 
	filter(min_dist < 3000) %>%
	left_join(
		is_pos_in_ref %>%
		filter(genome == "Query") %>%
		select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
		pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)),
		by = c("Line", "Gen", "is_cluster_id")
	) %>% filter(Gen == 2, dir, min_dist<100) %>%
	summarise(total = n(), simple = sum((sv_r == "simple_insertion") | (sv_f == "simple_insertion"))) %>%
	mutate(other = total - simple) 

total,simple,other
<int>,<int>,<int>
20,17,3


In [11]:
# those that are not simple insertions
excision_stat_df %>% select(Line:min_dist, flank_dist, min_start_raw, min_end_raw, dir) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # fixed 20240703 
	filter(min_dist < 3000) %>%
	left_join(
		is_pos_in_ref %>%
		filter(genome == "Query") %>%
		select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
		pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)),
		by = c("Line", "Gen", "is_cluster_id")
	) %>% filter(Gen == 2, dir) %>%
	filter(!(sv_r == "simple_insertion") & !(sv_f == "simple_insertion")) 

Line,Gen,is_cluster_id,min_dist,flank_dist,min_start_raw,min_end_raw,dir,position_status_r,position_status_f,sv_r,sv_f,flank_pos_r,flank_pos_f
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
L03-4,2,3,7,-7,1122383,1122377,TRUE,original,original,pristine,pristine,1110057,1113146
L08-4,2,1,7,-7,69480,69474,TRUE,original,original,pristine,pristine,69490,72580
L09-4,2,12,84,84,3852079,3852164,TRUE,new,new,unknown,unknown,3853394,3853316


L09-4-2-12 was an IS insertion in Gen2, which accompanied deletion, but the IS insertion nor the deletion was not detected in Gen3. Thus, the excision is a false positive.
L03-4	2	3	and L08-4	2	1	 seem to be loss of ISs that were preexisting in the ancestor.

In [12]:
excision_stat_df.stat2 <- excision_stat_df %>% select(Line:min_dist, flank_dist, min_start_raw, min_end_raw, dir) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # fixed 20240703 
	filter(min_dist < 100) %>%
	left_join(
		is_pos_in_ref %>%
		filter(genome == "Query") %>%
		select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
		pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)),
		by = c("Line", "Gen", "is_cluster_id")
	) %>% filter(Gen == 2, dir) %>%
	summarise(total = n(), simple = sum((sv_r == "simple_insertion") | (sv_f == "simple_insertion")),
		original = sum((position_status_r == "original") & (position_status_f == "original")),
	) %>%
	mutate(other = total - simple - original)
excision_stat_df.stat2

total,simple,original,other
<int>,<int>,<int>,<int>
20,17,2,1


In [13]:
is_pos_in_ref.stat <- is_pos_in_ref %>%
		filter(genome == "Query") %>%
		select(is_cluster_id = pos_id, Line, Gen, position_status, sv, flank_pos = pos, flank) %>%
		pivot_wider(names_from = "flank", values_from = c(position_status, sv, flank_pos)) %>%
	filter(!(Line == "L04-1" & Gen == 2)) %>% # fixed 20240703 
	filter(Gen == 2) %>%
	summarise(total = n(), simple = sum((sv_r == "simple_insertion") | (sv_f == "simple_insertion"), na.rm = TRUE),
		original = sum((position_status_r == "original") & (position_status_f == "original"), na.rm = TRUE),
	) %>%  # there seems to be NAs, NAs are formed when the flank match is too short due to two ISs being too close
	mutate(other = total - simple - original)
is_pos_in_ref.stat

total,simple,original,other
<int>,<int>,<int>,<int>
794,207,382,205


## STAT: Bionom.test new or not
The status of lost ends were often "simple". This indicates that the IS was inserted during the MA until 8th by simple insertions. 

In [14]:
binom.test(excision_stat_df.stat2$simple, sum(excision_stat_df.stat2), is_pos_in_ref.stat$simple/sum(is_pos_in_ref.stat), alternative="greater")


	Exact binomial test

data:  excision_stat_df.stat2$simple and sum(excision_stat_df.stat2)
number of successes = 17, number of trials = 40, p-value = 3.983e-06
alternative hypothesis: true probability of success is greater than 0.1303526
95 percent confidence interval:
 0.2918466 1.0000000
sample estimates:
probability of success 
                 0.425 
